In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
from sklearn.preprocessing import FunctionTransformer
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.pipeline import Pipeline
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/comp90051-2024s1-project-1/test_data.json
/kaggle/input/comp90051-2024s1-project-1/sample.csv
/kaggle/input/comp90051-2024s1-project-1/domain2_train_data.json
/kaggle/input/comp90051-2024s1-project-1/domain1_train_data.json


In [2]:
domain1_path = r"/kaggle/input/comp90051-2024s1-project-1/domain1_train_data.json" 
domain2_path = r"/kaggle/input/comp90051-2024s1-project-1/domain2_train_data.json"
domain3_path = r"/kaggle/input/comp90051-2024s1-project-1/test_data.json"

def sentence_to_vector(sentence, model):
    vectors = [model.wv[word] for word in sentence if word in model.wv]
    if vectors:
 
        mean_vector = np.mean(np.array(vectors), axis=0)

        return mean_vector.tolist()
    else:
        oov_words_list.extend([word for word in sentence if word not in model.wv])
        print(oov_words_list)
        return [0] * model.vector_size

def read_in_data(file_path):
    with open(file_path, 'r') as file:
        file_content = file.read()

    json_strings = file_content.strip().split('\n')
    data_list = []
    for json_string in json_strings:
        data = json.loads(json_string)
        data_list.append(data)
    df = pd.DataFrame(data_list)

    return df

In [3]:
df1 = read_in_data(domain1_path)
df2 = read_in_data(domain2_path)
df3 =  read_in_data(domain3_path)
df_combined= pd.concat([df1, df2], axis=0, ignore_index=True)
sentences = df_combined['text'].tolist()
av_len1 = df1['text'].apply(lambda x: len(x)).mean()
av_len2 = df2['text'].apply(lambda x: len(x)).mean()
av_len3 = df_combined['text'].apply(lambda x: len(x)).mean()
print("Domain 1 dataset has", len(df1), "instances with average length", av_len1)
print("Domain 2 dataset has", len(df2), "instances with average length", av_len2)
print("Domain combined dataset has", len(df_combined), "instances with average length", av_len3)
X = pd.DataFrame(df2[['text', 'id']])
y = pd.DataFrame(df2['label'])
rus = RandomUnderSampler(sampling_strategy={0: 2500})

X_undersampled, y_undersampled = rus.fit_resample(X, y)
df2_undersampled = pd.DataFrame(X_undersampled, columns=X.columns)
df2_undersampled['label'] = y_undersampled 
X = pd.DataFrame(df2_undersampled[['text', 'id']])
y = pd.DataFrame(df2_undersampled['label'])
ros = RandomOverSampler( random_state=0)
X_resampled, y_resampled = ros.fit_resample(X, y)
df2_resampled = pd.DataFrame(X_resampled, columns=X.columns)
df2_resampled['label'] = y_resampled
df_training= pd.concat([df1, df2_resampled], axis=0, ignore_index=True)
#df_testing= pd.concat([df1, df2], axis=0, ignore_index=True)
# Assuming df_training is your DataFrame containing the 'text' column
df_training['text'] = df_training['text'].apply(lambda x: [token for token in x if token != 0])

df_testing=df3
df_vocab_training=pd.concat([df_combined, df_testing], axis=0, ignore_index=True)

Domain 1 dataset has 5000 instances with average length 181.6522
Domain 2 dataset has 13000 instances with average length 264.12984615384613
Domain combined dataset has 18000 instances with average length 241.2193888888889


In [4]:
sentences_training = df_combined['text'].tolist()
Vocab_model = Word2Vec(sentences_training, vector_size=100, window=4, min_count=10, workers=4)
def sentence_to_vector(sentence, model):
    vectors = [model.wv[word] for word in sentence if word in model.wv]
    if vectors:
 
        mean_vector = np.mean(np.array(vectors), axis=0)
        return mean_vector.tolist()
    else:
        return [0] * model.vector_size 
def sentences_to_vectors(sentences, model):
    return np.array([sentence_to_vector(sentence, model) for sentence in sentences]).tolist()

def tfidf_preprocess(X):
    return [' '.join(map(str, doc)) for doc in X]

df_combined['text'] = df_combined['text'].apply(lambda x: ' '.join(map(str, x)))
token_pattern = r"\b\d+\b"
vectorizer = TfidfVectorizer(token_pattern=token_pattern)
vectorizer.fit(df_combined['text'])

TfidfVectorizer(token_pattern='\\b\\d+\\b')

In [5]:
pipeline_model1 = Pipeline([
    ('word2vec', FunctionTransformer(sentences_to_vectors, kw_args={'model': Vocab_model})),
    ('clf', RandomForestClassifier(n_estimators=100, random_state=42))
])
pipeline_model2 = Pipeline([
    ('preprocess', FunctionTransformer(tfidf_preprocess)),
    ('tfidf', vectorizer),
    ('svm',  SVC(C=1e6, kernel='rbf', gamma=1))
])

In [6]:
print("starting model training")
pipeline_model1.fit(df_training['text'], df_training['label'])
print("Random forest trained")
pipeline_model2.fit(df_training['text'], df_training['label'])
print("SVM trained")

starting model training
Random forest trained
SVM trained


In [7]:
stacking_clf = StackingClassifier(
    estimators=[
        ('model1', pipeline_model1),
        ('model2', pipeline_model2)
    ],
    final_estimator=LogisticRegression()
)

In [8]:
print("fiting stack")
stacking_clf.fit(df_training['text'], df_training['label'])
print("predicting classes ")
predictions = stacking_clf.predict(df_testing['text'])
print("predictions made")

fiting stack
predicting classes 
predictions made


In [9]:
output_df = pd.DataFrame({
    'id': df_testing['id'],  # Assuming 'id_column' contains the IDs
    'class': predictions # Assuming 'predictions' contains the predicted labels
})

output_csv_file = 'stack_forest_Bothfull.csv'


output_df.to_csv(output_csv_file, index=False)

print("Predictions saved to", output_csv_file)

Predictions saved to stack_forest_Bothfull.csv


In [10]:
def find_oov_words(test_data, model):
    oov_words = []
    for sentence in test_data:
        for word in sentence:
            if word not in model.wv:
                oov_words.append(word)
    return oov_words

oov_words_test = find_oov_words(df_testing['text'], Vocab_model)

# Print the list of out-of-vocabulary words
print("Out-of-vocabulary words in testing data:", oov_words_test)
print(len(oov_words_test))

Out-of-vocabulary words in testing data: [21776, 37325, 18144, 73872, 14513, 19809, 71013, 17771, 16114, 18009, 18030, 21084, 15594, 28052, 34796, 23805, 20205, 18704, 50909, 76224, 75402, 25804, 24526, 15288, 25804, 14689, 32624, 36546, 15288, 36546, 32624, 16647, 38106, 17414, 37220, 23106, 15796, 22209, 21385, 17064, 23913, 14524, 63389, 14524, 14524, 14524, 14524, 14524, 14524, 14524, 14524, 14524, 14524, 63389, 53057, 35107, 50984, 14524, 20152, 14524, 63389, 15279, 27861, 15279, 16954, 16779, 16954, 25236, 16779, 18876, 28257, 37424, 25658, 37424, 28257, 35178, 24894, 14883, 18710, 57914, 15376, 26407, 14717, 37424, 15613, 23271, 27720, 27686, 68727, 15573, 15338, 15338, 15338, 27142, 28257, 52909, 65760, 33996, 16913, 77452, 26089, 15541, 66890, 54651, 22031, 16602, 40586, 15014, 23085, 30889, 19493, 16872, 19865, 67215, 15895, 67759, 18337, 33997, 71700, 80070, 31890, 17618, 16439, 16386, 19345, 62452, 62451, 46870, 23260, 25173, 28340, 15460, 20725, 15940, 18740, 17748, 28621,